<a href="https://colab.research.google.com/github/farshidehkordi/Homework2_AI/blob/main/TP2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import os

# List files in the current directory
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
# Import libraries
import os
import pandas as pd
import logging
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision import datasets, transforms
from PIL import Image

# Classes are :
# 0 - Boston
# 1 - London
# 2 - Montreal
# 3 - Paris
# 4 - Quebec

# Define the network architecture (fully connected using ReLU) :
class ModifiedDropoutNetwork(nn.Module):
    def __init__(self, num_classes=5):
        super(ModifiedDropoutNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64)
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 28 * 28, 256)  # Adjust the input size of fc1
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

# Hyperparameters
num_epochs = 5
batch_size = 64
initial_learning_rate = 0.001
patience = 5

# Load dataset
train_data_path = '/content/drive/MyDrive/glo-7030-ou-suis-je-h2024/train/'
test_data_path = '/content/drive/MyDrive/glo-7030-ou-suis-je-h2024/test/'

# Define the transform
data_transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

class SingleClassTestDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = sorted(os.listdir(root_dir))  # Assuming the images are named as '00000.jpg', '00001.jpg', etc.

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, 0  # Assigning the class label as 0 for all images in the test dataset

# Splitting the training dataset into train and validation sets
train_dataset = datasets.ImageFolder(root=train_data_path, transform=data_transform)
valid_size = int(0.2 * len(train_dataset))  # For example, you can use 20% of the training data for validation
train_size = len(train_dataset) - valid_size  # Adjust train_size to exclude validation set
train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=SingleClassTestDataset(root_dir=test_data_path, transform=data_transform), batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model
model = ModifiedDropoutNetwork(num_classes=5).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=initial_learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=patience, factor=0.5)

# Training the model
train_loss_list = []
valid_loss_list = []
train_accuracy_list = []
valid_accuracy_list = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100 * correct / total
    train_loss_list.append(train_loss)
    train_accuracy_list.append(train_accuracy)

    # Validation phase
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_loss /= len(valid_loader)
    valid_accuracy = 100 * correct / total
    valid_loss_list.append(valid_loss)
    valid_accuracy_list.append(valid_accuracy)

    # Print training and validation loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Train Acc: {train_accuracy:.2f}, Valid Acc: {valid_accuracy:.2f}')

    # Step with the scheduler
    scheduler.step(valid_loss)

# Testing the model
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}')

# Get and print the last learning rate
last_lr = optimizer.param_groups[0]['lr']
print(f'Last learning rate: {last_lr}')

# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss_list, '-o', label='Train Loss')
plt.plot(valid_loss_list, '-o', label='Valid Loss')
plt.title('Training and Validation Loss Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotting the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy_list, '-o', label='Train Accuracy')
plt.plot(valid_accuracy_list, '-o', label='Valid Accuracy')
plt.title('Training and Validation Accuracy Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

Epoch [1/5], Train Loss: 1.6861, Valid Loss: 1.5558, Train Acc: 23.16, Valid Acc: 25.61
Epoch [2/5], Train Loss: 1.5616, Valid Loss: 1.5543, Train Acc: 28.03, Valid Acc: 28.54
